# DESaster Application Template
v. 10312017

## Required Modules

In [1]:
# System modules
import sys, random, inspect
from datetime import datetime
from IPython.display import display
import sys, inspect

# Data analysis modules
import pandas as pd
import numpy as np
from scipy.stats import uniform, norm, beta, weibull_min, rv_discrete
import random
from ipywidgets import Dropdown


# SimPy modules
import simpy
from simpy import Resource, Container, FilterStore
from simpy.util import start_delayed

# Viz modules
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import plugins
import branca.colormap as cm
from folium.plugins import MarkerCluster
from folium import Map, FeatureGroup, Marker, LayerControl

### DESaster Modules
desaster_path = "/Users/geomando/Dropbox/github/DESaster"
sys.path.append(desaster_path)
import desaster
from desaster.io import *
from desaster.structures import *
from desaster.financial import *
from desaster.technical import *
from desaster.entities import *
from desaster.policies import *
from desaster.visualize import dashboard, folium_map

## Import Households Attribute Data
Input data must be MS Excel .xlsx file with format (column names and sheet names) of ../inputs/desaster_input_data_template.xlsx

In [2]:
scenario_file = '../inputs/desaster_input_data_template.xlsx'

__Create Pandas dataframe of attribute data for all OwnerHouseholds to be modeled in the simulation.__

In [3]:
owners_df = pd.read_excel(scenario_file, sheetname='owners')
owners_df

,owner,owner_income,owner_savings,owner_insurance,owner_credit,address,monthly_cost,move_in_cost,occupancy,tenure,bedrooms,bathrooms,area,year_built,value,damage_state,listed,longitude,latitude
0,Alfred,60000,100000,0,700,62 That St,1482.755005,30690,Mobile Home,Owner Occupied,1,1,1100,1920,306900,Complete,False,-90.296127,43.224344
1,Bruce,1000000,100,1,700,720 This Rd,4043.877286,83700,Single Family Dwelling,Owner Occupied,4,3,3000,1920,837000,None,False,-90.295697,43.224219
2,Selena,45000,50000,0,700,1001 Other Ave,1010.969322,20925,Single Family Dwelling,Owner Occupied,2,1,750,1960,209250,None,False,-90.296706,43.223984
3,Fish,125000,100,0,700,26000 Out There Lane,2695.918191,55800,Single Family Dwelling,Owner Occupied,3,2,2000,2010,558000,None,False,-90.296642,43.223750
4,Jerome,100000000,100000000,1,700,100 New Ave,1449.418382,30000,Mobile Home,Owner Occupied,1,1,1100,1920,300000,None,True,-90.295054,43.224375
5,Barbara,100000000,100000000,1,700,101 New Ave,3865.115686,80000,Single Family Dwelling,Owner Occupied,4,3,3000,1920,800000,None,True,-90.294539,43.224391
6,Lucius,100000000,100000000,1,700,102 New Ave,1010.969322,20925,Single Family Dwelling,Owner Occupied,2,1,750,1960,209250,None,True,-90.295225,43.223953
7,Dick,100000000,100000000,1,700,103 New Ave,2415.697304,50000,Single Family Dwelling,Owner Occupied,3,2,2000,2010,500000,None,True,-90.295225,43.223718


__Create Pandas dataframe of attribute data for all RenterHouseholds to be modeled in the simulation.__

In [4]:
renters_df = pd.read_excel(scenario_file, sheetname='renters')
renters_df.head()

,tenant,address,occupancy,tenure,tenant_income,tenant_savings,tenant_credit,tenant_insurance,monthly_cost,move_in_cost,...,year_built,value,damage_state,listed,longitude,latitude,owner,owner_savings,owner_insurance,owner_credit
0,Ivy,262 That St,Mobile Home,Rental,119040,1000,700,0,2480,4960,...,1920,279000,Moderate,False,-90.294238,43.224015,Alice,30000,0,700
1,Edward,4720 This Rd,Single Family Dwelling,Rental,178560,1000,700,0,3720,7440,...,1920,418500,None,False,-90.293766,43.224062,Julie,30000,0,700
2,Oswald,2301 Other Ave,Single Family Dwelling,Rental,1000000,1000000,700,0,1240,2480,...,1960,139500,None,False,-90.293294,43.224125,Gerry,30000,1,700
3,James,74000 Out There Lane,Single Family Dwelling,Rental,238080,1000000,700,0,4960,9920,...,2010,558000,None,False,-90.293058,43.223937,Sally,30000,1,700
4,Butch,100 Old Ave,Mobile Home,Rental,100000000,1000,700,0,2480,4960,...,1920,279000,None,True,-90.293080,43.223671,Greg,100000000,1,700


# Inspect DESaster library modules and classes

Choose DESaster module to list classes (next cell)

In [5]:

modules = sorted(str(name) for name, obj in inspect.getmembers(desaster) if inspect.ismodule(obj))
module_chooser = Dropdown(
    options=modules
)
display(module_chooser)

List classes in module selected above

In [6]:
print('DESaster ' + str(module_chooser.value).title() + ' Classes')
print('--------------')
classes = sorted(str(name) for name, obj in inspect.getmembers(eval('desaster.' + module_chooser.value)) if inspect.isclass(obj))
for i in classes: print(i)

DESaster Entities Classes
--------------
Building
Container
Household
Landlord
Owner
OwnerHousehold
RenterHousehold
ResidentialBuilding


In [7]:
methods = sorted(str(name) for name, obj in inspect.getmembers(desaster.entities.OwnerHousehold) if inspect.isfunction(obj))
for i in methods: print(i)

__init__
buy_home
occupy_permanent
rent_home
story_to_text
writeAttributes
writeGaveUpHomeBuySearch
writeGaveUpHomeRentSearchStock
writeHomeBuy
writeHomeRent
writeOccupy
writeOwnsOccupies
writeResides
writeStartHomeBuySearch
writeStartHomeRentSearch
writeVacated


## Setup the Simulation Environment, Define Parameter Distributions, and Populate with Households, Recovery Programs, and Recovery Policies

__Set Simpy simulation environment__

In [8]:
env = simpy.Environment()

__Indicate whether the simulation will keep track of the stories of each entity in the simulation. This can also be set individually for each group of imported entities (e.g., each call to an entities.Entity class or subclass.__

In [9]:
write_story = True

__Define the probability distributions that can be used in the simulation to define various recovery program process event durations.__

In [10]:
fema_ia_dist = norm(loc = 10, scale = 0)
sba_home_sba_dist = norm(loc = 10, scale = 0)
sba_biz_sba_dist = norm(loc = 10, scale = 0)
insurance_dist = norm(loc = 10, scale = 0)
start_delay_dist = norm(loc = 10, scale = 0)
inspection_dist = norm(loc = 0.5, scale = 0)
assessment_dist = norm(loc = 10, scale = 0)
permitting_dist = norm(loc = 10, scale = 0)
repair_dist = norm(loc = 10, scale = 0)
demolition_dist = norm(loc = 10, scale = 0)
occupy_dist = norm(loc = 10, scale = 0)
rent_home_dist = norm(loc = 10, scale = 0)
buy_home_dist = norm(loc = 10, scale = 0)

declaration = norm.rvs(loc = 30, scale = 0) # Placeholder deterministic scalar that defines number of days before federal disaster declared
sba_deadline = norm.rvs(loc = 60, scale = 0) # Typically duration in days after declaration that SBA applications must be submitted

__Instantiate the recovery program objects that will be used in the simulation. Each recovery program requires
specification of a duration probability distribution (set above). Unlike shown below, the distributions do not have to be the same for each program. Currently all are the same scalars to simplify interpretation and debugging.__

In [11]:
# Instantiate a FEMA Individual Assistance program
fema_ia = HousingAssistanceFEMA(env, staff=100, budget=10000000, duration=fema_ia_dist, max_outlay=30000,
                                  deadline=540, declaration=declaration)
# Instantiate a SBA Loan program to use w/ OwnerHouseholds
sba_home_loan = RealPropertyLoanSBA(env, officers=10, inspectors=10, duration=sba_home_sba_dist, max_loan = 200000, 
                            min_credit = 600, deadline=sba_deadline, declaration=declaration)
# Instantiate a SBA Loan program to use w/ Landlords (higher maximum for businesses)
sba_biz_loan = RealPropertyLoanSBA(env, officers=10, inspectors=10, duration=sba_biz_sba_dist, 
                           max_loan = 2000000, deadline=sba_deadline, declaration=declaration) # SBA max biz loan = $2mil
# Instantiate a private insurance program
insurance = OwnersInsurance(env, staff=100, deductible=0.0, duration=insurance_dist)
# Instantiate a rapid inspeaction (tagging) program
inspection = InspectionProgram(env, staff=1000, duration=inspection_dist)
# Instantiate a detailed engineering assessment program
assessment = EngineeringAssessment(env, staff=1000, duration=assessment_dist)
# Instantiate a building permit processing program
permitting = PermitProgram(env, staff=1000, duration=permitting_dist) 
# Instantiate a program to represent private repair contractors
repair = RepairProgram(env, staff=1000, materials=1000000000, duration=repair_dist)
# Instantiate a program to represent private demolition contractors
demolition = DemolitionProgram(env, staff=1000, duration=demolition_dist)

__Instantiate any recovery policies. Policies are custom classes that define different recovery program arrangements, sequences, logic, patience, etc. Currently only one is written. Policy classes also provides an example of how to model entity "patience" and process interuption.__

In [12]:
insurance_ia_sba_seq = Insurance_IA_SBA_Sequential(env)
insurance_ia_sba_para = Insurance_IA_SBA_Parallel(env)
insurance_firstthen_ia_sba_para = Insurance_FirstThen_IA_SBA_Parallel(env)
insurance_sba_seq = Insurance_SBA_Sequential(env)
insurance_sba_para = Insurance_SBA_Parallel(env)
repair_stock = RepairVacantBuilding(env)

__Create empty Simpy FilterStores to use as housing stocks.__

In [13]:
owner_occupied_stock = FilterStore(env)  # To put the residences associated with owners
rental_stock = FilterStore(env) # To put the residences associated with renters
temporary_stock = FilterStore(env) # To put the temporary shelters and housing 

In [14]:
vacant_owner_stock_df = pd.read_excel(scenario_file, sheetname='forsale_stock')
importResidentialBuildingStock(env, vacant_owner_stock_df, owner_occupied_stock)
vacant_owner_stock_df.head()

,address,monthly_cost,move_in_cost,occupancy,tenure,bedrooms,bathrooms,area,year_built,value,damage_state,listed,longitude,latitude,owner,owner_income,owner_savings,owner_insurance,owner_credit
0,100 New Ave,1449.418382,30000,Mobile Home,Owner Occupied,1,1,1100,1920,300000,None,True,-90.295054,43.224375,Jerome,100000000,100000000,1,850
1,101 New Ave,3865.115686,80000,Single Family Dwelling,Owner Occupied,4,3,3000,1920,800000,None,True,-90.294539,43.224391,Barbara,100000000,100000000,1,850
2,102 New Ave,1010.969322,20925,Single Family Dwelling,Owner Occupied,2,1,750,1960,209250,None,True,-90.295225,43.223953,Lucius,100000000,100000000,1,850
3,103 New Ave,2415.697304,50000,Single Family Dwelling,Owner Occupied,3,2,2000,2010,500000,None,True,-90.295225,43.223718,Dick,100000000,100000000,1,850


In [15]:
vacant_rental_stock_df = pd.read_excel(scenario_file, sheetname='forrent_stock')
importResidentialBuildingStock(env, vacant_rental_stock_df, rental_stock)
vacant_rental_stock_df.head()

,address,occupancy,tenure,monthly_cost,move_in_cost,bedrooms,bathrooms,area,year_built,value,damage_state,listed,longitude,latitude,owner,owner_savings,owner_insurance,owner_credit
0,100 Old Ave,Mobile Home,Rental,2480,4960,1,1,1000,1920,279000,None,False,-90.293080,43.223671,Greg,100000000,1,850
1,101 Old Ave,Single Family Dwelling,Rental,3720,7440,3,2,1500,1920,418500,None,False,-90.292736,43.223546,Allison,100000000,1,850
2,102 Old Ave,Single Family Dwelling,Rental,1240,2480,0,1,500,1960,139500,None,False,-90.296191,43.223406,Rachel,100000000,1,850
3,103 Old Ave,Single Family Dwelling,Rental,4960,9920,2,2,2000,2010,558000,None,False,-90.295740,43.223359,Larry,100000000,1,850
4,341 Where St,Mobile Home,Rental,1,10,1,1,1100,1920,306900,None,False,-90.294238,43.224015,Gigi,100000000,1,850


In [16]:
temp_stock_df = pd.read_excel(scenario_file, sheetname='temp_stock')
importResidentialBuildingStock(env, temp_stock_df, rental_stock)
temp_stock_df.head()

,address,occupancy,tenure,monthly_cost,move_in_cost,bedrooms,bathrooms,area,year_built,value,damage_state,listed,longitude,latitude,owner,owner_savings,owner_insurance,owner_credit
0,100 Old Ave,Mobile Home,Rental,2480,4960,1,1,1000,1920,279000,None,True,-90.293080,43.223671,Greg,100000000,1,700
1,101 Old Ave,Single Family Dwelling,Rental,3720,7440,3,2,1500,1920,418500,None,True,-90.292736,43.223546,Allison,100000000,1,700
2,102 Old Ave,Single Family Dwelling,Rental,1240,2480,0,1,500,1960,139500,None,True,-90.296191,43.223406,Rachel,100000000,1,700
3,103 Old Ave,Single Family Dwelling,Rental,4960,9920,2,2,2000,2010,558000,None,True,-90.295740,43.223359,Larry,100000000,1,700
4,341 Where St,Mobile Home,Rental,1,10,1,1,1100,1920,306900,None,True,-90.294238,43.224015,Gigi,30000,0,700


__Create a list of OwnerHousehold objects based on input data. Place each associated residence in the rented housing stock.__

In [17]:
owners = importEntities(env, owners_df, 'OwnerHousehold', owner_occupied_stock, write_story)

__Create a list of RenterHousehold objects based on input data. Place each associated residence in the rented housing stock.__

In [18]:
renters = importEntities(env, renters_df, 'RenterHousehold', rental_stock, write_story)

# Specify master processes for Landlords, OwnerHouseholds, and Renter Households
Note: Master processes define custom, case-specific, scenario-based, or user-defined logic for modeled entities. They are not "hard coded" processes or policies in DESaster.

*A listing of arguments for each of the recovery programs and policies to help write master processes.*

In [19]:
def list_args(programs_policies):
    for p in programs_policies:
        print(str(p.__func__)[10:-16], inspect.getfullargspec(p).args) # Useful to determine the arguments for the recovery programs

programs_policies = [fema_ia.process, sba_home_loan.process, sba_biz_loan.process, insurance.process, 
                     inspection.process, assessment.process, permitting.process, repair.process, 
                     demolition.process, insurance_ia_sba_seq.policy, insurance_ia_sba_para.policy, 
                     insurance_firstthen_ia_sba_para.policy, insurance_sba_seq.policy, insurance_sba_para.policy, 
                     repair_stock.policy]

list_args(programs_policies)

HousingAssistanceFEMA.process ['self', 'entity', 'callbacks']
RealPropertyLoanSBA.process ['self', 'entity', 'callbacks']
RealPropertyLoanSBA.process ['self', 'entity', 'callbacks']
OwnersInsurance.process ['self', 'entity', 'callbacks']
InspectionProgram.process ['self', 'structure', 'entity', 'callbacks']
EngineeringAssessment.process ['self', 'structure', 'entity', 'callbacks']
PermitProgram.process ['self', 'structure', 'entity', 'callbacks']
RepairProgram.process ['self', 'structure', 'entity', 'callbacks']
DemolitionProgram.process ['self', 'structure', 'entity', 'callbacks']
Insurance_IA_SBA_Sequential.policy ['self', 'insurance_program', 'fema_program', 'sba_program', 'entity', 'search_patience']
Insurance_IA_SBA_Parallel.policy ['self', 'insurance_program', 'fema_program', 'sba_program', 'entity', 'search_patience']
Insurance_FirstThen_IA_SBA_Parallel.policy ['self', 'insurance_program', 'fema_program', 'sba_program', 'entity', 'search_patience']
Insurance_SBA_Sequential.polic

__A really basic custom master process as an example of how to create one.__

In [20]:
def basic_process(inspection_program, assessment_program, permit_program, repair_program, entity):
    yield env.process(inspection_program.process(entity.property, entity))
    yield env.process(assessment_program.process(entity.property, entity))
    yield env.process(permit_program.process(entity.property, entity))
    yield env.process(repair_program.process(entity.property, entity))

__A custom master process for Landlords. Landlords are the owners of renters' residences and so are the ones to seek financial assistance for repairs.__

In [21]:
def landlord_process(env, insurance_program, loan_program, 
                 assessment_program, permit_program, demolish_program, repair_program, entity):
    
    money_patience = 100000  # days until give up the search for repair money
    
    # Simulate damaged properties
    if entity.property.damage_state != 'None': 
         
        # Landlord search for financial assistance using an Insurance_SBA policy. Note two alternate versions 
        # can be used: insurance_sba_para or insurance_sba_seq. Paste in the desired policy approach below.
        yield env.process(insurance_sba_seq.policy(insurance_program, loan_program, entity, money_patience)) # Sequential

        # If landlord gives up looking for recovery funds, evict their tenant
        if entity.gave_up_funding_search != None:
            entity.evict_tenant()
            
            if entity.write_story:
                entity.story.append(
                '{0} decided not to repair their {1}. '.format(
                entity.name, entity.property.occupancy.lower()
                                                                )
                                    )
            return
        
        # If has enough recovery funds, repair; if not, evict tenant.
        if entity.recovery_funds.level >= entity.property.damage_value:
            yield env.process(assessment_program.process(entity.property, entity))
            yield env.process(permit_program.process(entity.property, entity))
            
            # Demolish property if > extensive damage
            if entity.property.damage_state == 'Extensive' or entity.property.damage_state == 'Complete':
                yield env.process(demolish_program.process(entity.property, entity))
            yield env.process(repair_program.process(entity.property, entity))
        else:
            if entity.tenant.residence != None:
                entity.evict_tenant()  

__A custom master process for OwnerHouseholds (owner occupiers). Don't do anything if no damage suffered. If residence damage is "Complete", abandon home and look to buy a different one. Otherwise look for financial assistance for repairs. If money for repairs can't be found (patience runs out), look for a new home. If home search patience runs out, simply stop.__

In [22]:
def owner_process(env, inspection_program, insurance_program, fema_program, loan_program, 
                 assessment_program, permit_program, demolish_program, rebuild_program, forsale_stock, 
                  forrent_stock, entity):
    
    money_patience = 200000  # days until give up the search for rebuild money
    home_patience = 15000  # days until give up the search for a new home
    
    # Do inspections after inspectors are mobilized
    yield env.timeout(start_delay_dist.rvs())
    yield env.process(inspection_program.process(entity.property, entity))
    
    # Process damaged properties
    if entity.property.damage_state == 'None':
        yield env.process(entity.occupy_permanent(duration = norm(loc = 0, scale = 0)))   
    else:
        find_money = env.process(insurance_firstthen_ia_sba_para.policy(insurance_program, fema_program,
                                                   loan_program, entity, money_patience))
        find_rental = env.process(entity.rent_home(forrent_stock, rent_home_dist,
                                                   occupancy_list = ['single family dwelling', 'mobile home', 'temporary lodging'],
                                               search_patience = home_patience, rooms_tol = -2,
                                                   area_pct = 0.1, housing_ratio = 1.0, permanent = False))
        if entity.property.damage_state == 'Extensive' or entity.property.damage_state == 'Complete':
            yield find_rental | find_money
               
        # Homeowner search for financial assistance using an Insurance_SBA policy. Note two alternate versions 
        # can be used: insurance_ia_sba_para, insurance_ia_sba_seq, and insurance_firstthen_ia_sba_para. 
        # Paste in the desired policy approach below.
        else:
            yield find_money 

        # If not enough money to repair home or home completely damaged, search for a new home to purchase.
        if (entity.recovery_funds.level < entity.property.damage_value or
                entity.property.damage_state == 'Complete'):
            yield env.process(entity.buy_home(forsale_stock, buy_home_dist, down_payment_pct = 0.10,
                                                  search_patience = home_patience))
            
            if entity.gave_up_home_buy_search == None:
                yield env.process(entity.occupy_permanent(duration = occupy_dist))
            
            return
        # Otherwise repair home.       
        elif entity.recovery_funds.level >= entity.property.damage_value:
            
            yield env.process(assessment_program.process(entity.property, entity))
            yield env.process(permit_program.process(entity.property, entity))
            if entity.property.damage_state == 'Extensive' or entity.property.damage_state == 'Complete':
                yield env.process(demolish_program.process(entity.property, entity))
            yield env.process(rebuild_program.process(entity.property, entity))    
            yield env.process(entity.occupy_permanent(duration = occupy_dist))     

__A custom master process for RenterHouseholds. For the most part it simply initiates a process for their landlords. If they are evicted by their landlords, the renter will look for a new home. If home search patience runs out, simply stop. Otherwise, occupy home after landlord repairs it.__

In [23]:
def renter_process(env, landlord_process, inspection_program, search_stock, entity):
        
    home_patience = 550  # days until give up the search for a new home
    
    # Do inspection after inspectors are mobilized
    yield env.timeout(start_delay_dist.rvs())
    yield env.process(inspection_program.process(entity.landlord.property, entity.landlord)) 
      
    if entity.residence.damage_state == 'None':
        yield env.process(entity.occupy_permanent(duration = norm(loc = 0, scale = 0)))
    else:
         # If is extensively/completely damaged, evict tenant.
        if entity.residence.damage_state == 'Extensive' or entity.residence.damage_state == 'Complete':
            yield env.process(entity.landlord.evict_tenant())

            rent_home_start = env.process(entity.rent_home(search_stock, rent_home_dist,
                                                           search_patience = home_patience))
            
            yield rent_home_start | env.process(landlord_process)

            if rent_home_start.processed and entity.residence != None:
                yield env.process(entity.occupy_permanent(duration = occupy_dist))

        else:
            yield env.process(landlord_process)
            yield env.process(entity.occupy_permanent(duration = occupy_dist))
    
    entity.story.append(entity.landlord.story)


__Initiate the master process for each owner to be modeled in the simulation.__

In [24]:
#inspect.getfullargspec(owner_process).args # Useful to determine what arguments are required for the process.

for i in range(len(owners)):
    env.process(owner_process(env, inspection, insurance, fema_ia, sba_home_loan, 
                                    assessment, permitting, demolition, repair,
                                    owner_occupied_stock, rental_stock, owners[i]))

__Initiate the master process for each renter to be modeled in the simulation.__

In [25]:
# # #inspect.getfullargspec(renter_process).args # Useful to determine what arguments are required for the process.


for i in range(len(renters)):
    landlord_start = landlord_process(env, insurance, sba_biz_loan, assessment, permitting, 
                                    demolition, repair, renters[i].landlord)
    
    env.process(renter_process(env, landlord_start, inspection, rental_stock, renters[i]))

## Run the simulation

In [26]:
env.run()

In [27]:
i = 0
try:
    print('PROPERTIES: ', owners[i].name, ':', owners[i].prior_properties[0].address,
                      '-->', owners[i].property.address)
except:
    print('PROPERTIES: ', owners[i].name, ':', owners[i].property.address, '--> None')
    

try:
    print('RESIDENCES: ', owners[i].name, ':', owners[i].prior_residences[0].address,
          '-->', owners[i].prior_residences[1].address,
         '-->', owners[i].residence.address)
except:
    try:
        print('RESIDENCES: ', owners[i].name, ':', owners[i].prior_residences[0].address,
              '-->', owners[i].residence.address)
    except:
        print('RESIDENCES: ', owners[i].name, ':', owners[i].residence.address,
          '--> None')

PROPERTIES:  Alfred : 62 That St --> 100 New Ave
RESIDENCES:  Alfred : 62 That St --> 12303 Lodging Rd --> 100 New Ave


In [28]:
for item in owner_occupied_stock.items:
    print(item.owner.name, ',', item.address, ',', item.damage_state, ',', item.listed)

Barbara , 101 New Ave , None , True
Lucius , 102 New Ave , None , True
Dick , 103 New Ave , None , True
Bruce , 720 This Rd , None , False
Selena , 1001 Other Ave , None , False
Fish , 26000 Out There Lane , None , False
Jerome , 100 New Ave , None , True
Barbara , 101 New Ave , None , True
Lucius , 102 New Ave , None , True
Dick , 103 New Ave , None , True
Alfred , 62 That St , Complete , True
Alfred , 100 New Ave , None , False


In [29]:
for item in rental_stock.items:
    try:
        print('Owner:', item.owner.name, '; Renter:', item.owner.tenant.name, item.address, ',', item.damage_state, ',', item.listed)
    except:
        print('Owner:', item.owner.name, '; Renter: None', ',', item.address, item.damage_state, ',', item.listed)
    

Owner: Greg ; Renter: None , 100 Old Ave None , False
Owner: Allison ; Renter: None , 101 Old Ave None , False
Owner: Rachel ; Renter: None , 102 Old Ave None , False
Owner: Larry ; Renter: None , 103 Old Ave None , False
Owner: Gigi ; Renter: None , 341 Where St None , False
Owner: Blake ; Renter: None , 9900 Nowhere St None , False
Owner: Best Western ; Renter: None , 12300 Lodging Rd None , False
Owner: Best Western ; Renter: None , 12301 Lodging Rd None , False
Owner: Best Western ; Renter: None , 12302 Lodging Rd None , False
Owner: Greg ; Renter: None , 100 Old Ave None , True
Owner: Allison ; Renter: None , 101 Old Ave None , True
Owner: Rachel ; Renter: None , 102 Old Ave None , True
Owner: Larry ; Renter: None , 103 Old Ave None , True
Owner: Gigi ; Renter: None , 341 Where St None , True
Owner: Blake ; Renter: None , 9900 Nowhere St None , True
Owner: Best Western ; Renter: None , 12300 Lodging Rd None , True
Owner: Best Western ; Renter: None , 12301 Lodging Rd None , True
O

# Summarize and visualize simulation outputs

## OwnerHousehold summary statistics

# ****** `output_summary()` NEEDS TO BE FIXED TO ACCOUNT FOR PRIOR RESIDENCES ******

In [30]:
output_summary(owners, 'OwnerHousehold')

1 out of 8 owners suffered damage to their homes.
 0 out of 8 owners rebuilt or repaired their damaged home.
 0 out of 8 owners gave up searching for money.
 2 out of 8 owners searched to buy a new home.
 2 out of 8 owners bought a new home.
 0 out of 8 owners searched to temporarily rent a new home.
 0 out of 8 owners rented a temporary home.
 0 out of 8 owners gave up searching to buy a home. 0 out of 8 owners gave up searching to rent a temporary home. 0 out of 8 owners are homeless.



## Print OwnerHousehold stories

In [31]:
owners[0].story

['Alfred has $100000 of savings and a credit score of 700. ',
 'Alfred resides at 62 That St. ',
 'Alfred owns and occupies a 1 bedroom mobile home worth $306,900. ',
 "Alfred's mobile home was inspected 10 days after the event. ",
 'It was found to have a damage level of complete and was collapsed. ',
 'The value of the damage was $306,900. ',
 'Alfred started searching for a mobile home rental 10 days after the event. ',
 'Alfred has no hazard insurance. ',
 'Alfred vacated the owner occupied mobile home at 62 That St. ',
 'On day 10, Alfred found a temporary lodging hotel owned by Best Western at 12303 Lodging Rd with a monthly rent of $3,000 and move in cost of $60. ',
 'Alfred started searching to buy a mobile home 10 days after the event. ',
 'Alfred vacated the hotel temporary lodging at 12303 Lodging Rd. ',
 'On day 20, Alfred purchased a $300,000 mobile home at 100 New Ave with a down payment of $30,000. ',
 'Alfred applied for a $200,000 SBA loan 30.0 days after the event.',


In [32]:
owners[1].story

['Bruce has $100 of savings and a credit score of 700. ',
 'Bruce resides at 720 This Rd. ',
 'Bruce owns and occupies a 4 bedroom single family dwelling worth $837,000. ',
 "Bruce's single family dwelling was inspected 10 days after the event. ",
 'It was found to have a damage level of none and was functional. ',
 'The value of the damage was $0. ',
 'Bruce reoccupied the owner occupied single family dwelling at 720 This Rd 10 days after the event. ']

In [33]:
owners[2].story

['Selena has $50000 of savings and a credit score of 700. ',
 'Selena resides at 1001 Other Ave. ',
 'Selena owns and occupies a 2 bedroom single family dwelling worth $209,250. ',
 "Selena's single family dwelling was inspected 10 days after the event. ",
 'It was found to have a damage level of none and was functional. ',
 'The value of the damage was $0. ',
 'Selena reoccupied the owner occupied single family dwelling at 1001 Other Ave 10 days after the event. ']

In [34]:
owners[3].story

['Fish has $100 of savings and a credit score of 700. ',
 'Fish resides at 26000 Out There Lane. ',
 'Fish owns and occupies a 3 bedroom single family dwelling worth $558,000. ',
 "Fish's single family dwelling was inspected 10 days after the event. ",
 'It was found to have a damage level of none and was functional. ',
 'The value of the damage was $0. ',
 'Fish reoccupied the owner occupied single family dwelling at 26000 Out There Lane 10 days after the event. ']

## RenterHousehold summary statistics

In [35]:
output_summary(renters, 'RenterHousehold')

1 out of 8 renters' homes suffered damage.
 0 out of 8 renters are homeless.
 0 out of 8 renters relocated.
 1 out of 8 landlords' damaged property was rebuilt or repaired.
 0 landlords gave up searching for repair money.


## Print RenterHousehold stories

In [ ]:
renters[0].story

In [ ]:
renters[1].story

In [ ]:
renters[2].story

In [ ]:
renters[2].story

In [ ]:
df = households_to_df(owners)

In [ ]:
df['name']

In [ ]:
# outfile = '../outputs/' + str(datetime.now().hour) + str(datetime.now().minute) + str(datetime.now().day) \
#                  + str(datetime.now().month) + str(datetime.now().year) + '.csv'
    
# df.to_csv(outfile)

In [ ]:
event_list=[]

for i in df.columns:
    if "get" in i or "put" in i or "stop" in i or "start" in i or "name" in i or "gave" in i:
        event_list.append(i)

event_df = df[event_list]
event_df = event_df.set_index('name')

In [ ]:
event_df

__Visualize events experienced by an individual HomeOwner entity.__

In [ ]:
names = sorted([f for f in df['name']])

name_chooser = Dropdown(
    options=names
)
display(name_chooser)

In [ ]:
name = name_chooser.value

name_row = df[df['name']==name][['inspection_put',
 'inspection_get',
 'claim_put',
 'claim_get',
 'fema_put',
 'fema_get',
 'sba_put',
 'sba_get',
'occupy_put',
'occupy_get']]

%matplotlib inline
plt.figure(figsize=(10,10))
sns.set_style(style="whitegrid")
sns.set(font_scale=2)
ax = sns.stripplot(name_row.iloc[0], name_row.columns, jitter=True, size = 15, linewidth=1)
ax.set(xlabel="Days After Event", ylabel="Housing Recovery Events for {0}".format(name))
ax.set_xbound(lower=0)

Map of outputs

In [ ]:
# outfile = '../outputs/' + 'folium_map_' + str(datetime.now().hour) + str(datetime.now().minute) + str(datetime.now().day) \
#                  + str(datetime.now().month) + str(datetime.now().year) + '.html'
    
# folium_map(df, outfile = outfile)

Dashboard of outputs

In [ ]:
# outfile = '../outputs/' + 'dashboard_' + str(datetime.now().hour) + str(datetime.now().minute) + str(datetime.now().day) \
#                  + str(datetime.now().month) + str(datetime.now().year) + '.html'

# dashboard(df, outfile = outfile)